In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
from torch.nn.modules.conv import Conv2d
from torchvision import transforms

/Users/ishaan/miniconda3/envs/VisualSearch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/ishaan/miniconda3/envs/VisualSearch/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/ishaan/miniconda3/envs/VisualSearch/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <8E6D6BF5-9658-33B9-9D3C-DF587B2F99E7> /Users/ishaan/miniconda3/envs/VisualSearch/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
class IVSN(nn.Module):
  def __init__(self, model):
      super(IVSN, self).__init__()
      self.features = nn.Sequential(*list(model.children())[0][:30])
      for param in self.features.parameters():
        param.requires_grad_ = False

  def forward(self, x):
      x = self.features(x)
      return x

ConvSize, NumTemplates, Mylayer = 1, 512, 31
TotalTrials, targetsize, stimulisize = 240, (640, 480), (1280, 1024)
MMconv = Conv2d(NumTemplates, 1, kernel_size = (ConvSize, ConvSize), stride = (1, 1), padding = (1, 1))
# 512 input channels, 1 output channel, 1x1 kernel, stride 1, padding 1

In [4]:
from torchvision.models import VGG16_Weights

model_vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', weights=VGG16_Weights.DEFAULT)
model_ivsn = IVSN(model_vgg)
model_ivsn

Using cache found in /Users/ishaan/.cache/torch/hub/pytorch_vision_v0.10.0


IVSN(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [9]:
import importlib
import naturalSceneDataset

input_images = naturalSceneDataset.NaturalSceneDataset('data/naturaldesign/gt', 'data/naturaldesign/stimuli', 'data/naturaldesign/target', normalize_means=[0.485, 0.456, 0.406], normalize_stds=[0.229, 0.224, 0.225])

In [10]:
print(input_images[0])

(tensor([[[0.2078, 0.2078, 0.2118,  ..., 0.7059, 0.8118, 0.9176],
         [0.2000, 0.2000, 0.1882,  ..., 0.7137, 0.8549, 0.9765],
         [0.1725, 0.1686, 0.1529,  ..., 0.8196, 0.9490, 0.9882],
         ...,
         [0.0667, 0.0941, 0.1255,  ..., 0.4078, 0.5608, 0.6275],
         [0.0667, 0.1059, 0.1216,  ..., 0.4039, 0.5176, 0.5922],
         [0.0706, 0.1176, 0.1294,  ..., 0.3686, 0.4784, 0.6118]],

        [[0.1686, 0.1647, 0.1686,  ..., 0.4784, 0.6196, 0.7490],
         [0.1608, 0.1569, 0.1569,  ..., 0.4980, 0.6667, 0.8039],
         [0.1412, 0.1373, 0.1255,  ..., 0.6000, 0.7333, 0.7804],
         ...,
         [0.0000, 0.0157, 0.0314,  ..., 0.7804, 0.8431, 0.8039],
         [0.0000, 0.0196, 0.0275,  ..., 0.7843, 0.8000, 0.7765],
         [0.0000, 0.0314, 0.0353,  ..., 0.7490, 0.7725, 0.7961]],

        [[0.0706, 0.0784, 0.0980,  ..., 0.2902, 0.4941, 0.6745],
         [0.0627, 0.0706, 0.0824,  ..., 0.3451, 0.5804, 0.7686],
         [0.0510, 0.0549, 0.0549,  ..., 0.5098, 0.7137, 0